In [ ]:
import pandas as pd  # For timestamp in text files

In [ ]:
def save_extracted_text_to_files(
    pdf_path: str = "Data/HSC26-Bangla1st-Paper.pdf",
    start_page: Optional[int] = None,
    end_page: Optional[int] = None,
    dpi: int = 300,
    output_dir: str = "extracted_text"
):
    """
    Extract text from PDF using OCR and save to individual .txt files
    
    Args:
        pdf_path: Path to the PDF file
        start_page: Starting page number (0-indexed). If None, starts from beginning
        end_page: Ending page number (0-indexed, inclusive). If None, goes to end
        dpi: DPI for image conversion
        output_dir: Directory to save text files
    """
    
    # Create output directory if it doesn't exist
    import os
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"📁 Created directory: {output_dir}")
    
    # Open PDF to get page count
    doc = fitz.open(pdf_path)
    total_pages = len(doc)
    doc.close()
    
    # Set page range
    start_page = start_page if start_page is not None else 0
    end_page = end_page if end_page is not None else total_pages - 1
    
    # Validate page range
    start_page = max(0, min(start_page, total_pages - 1))
    end_page = max(start_page, min(end_page, total_pages - 1))
    
    print(f"💾 Saving extracted text from pages {start_page + 1} to {end_page + 1}")
    print(f"📄 PDF has {total_pages} total pages")
    
    # Get base filename for naming
    base_name = os.path.splitext(os.path.basename(pdf_path))[0]
    
    # Extract and save text from each page
    all_text = []
    successful_pages = []
    
    for page_num in range(start_page, end_page + 1):
        print(f"Processing page {page_num + 1}/{total_pages}...")
        
        # Extract text using OCR
        raw_text = extract_text_with_ocr(pdf_path, page_num, dpi)
        cleaned_text = preprocess_bengali_text(raw_text)
        
        if cleaned_text.strip():
            # Save individual page text
            page_filename = f"{base_name}_page_{page_num + 1:03d}.txt"
            page_filepath = os.path.join(output_dir, page_filename)
            
            with open(page_filepath, 'w', encoding='utf-8') as f:
                f.write(f"Page {page_num + 1} - Raw Text:\n")
                f.write("=" * 50 + "\n")
                f.write(raw_text)
                f.write("\n\n" + "=" * 50 + "\n")
                f.write(f"Page {page_num + 1} - Cleaned Text:\n")
                f.write("=" * 50 + "\n")
                f.write(cleaned_text)
                f.write("\n\n" + "=" * 50 + "\n")
            
            all_text.append(f"=== PAGE {page_num + 1} ===\n{cleaned_text}\n")
            successful_pages.append(page_num + 1)
            
            print(f"✅ Saved: {page_filename} ({len(cleaned_text)} characters)")
        else:
            print(f"⚠️ Page {page_num + 1}: No text extracted, skipping file creation")
    
    # Save combined text file
    if all_text:
        combined_filename = f"{base_name}_combined_pages_{start_page + 1}-{end_page + 1}.txt"
        combined_filepath = os.path.join(output_dir, combined_filename)
        
        with open(combined_filepath, 'w', encoding='utf-8') as f:
            f.write(f"Combined Text from {pdf_path}\n")
            f.write(f"Pages: {start_page + 1} to {end_page + 1}\n")
            f.write(f"Extraction Date: {pd.Timestamp.now()}\n")
            f.write(f"DPI: {dpi}\n")
            f.write(f"Total Pages Processed: {len(successful_pages)}\n")
            f.write("=" * 80 + "\n\n")
            
            for text in all_text:
                f.write(text)
                f.write("\n" + "-" * 80 + "\n\n")
        
        print(f"\n📄 Combined file saved: {combined_filename}")
        print(f"📊 Successfully processed {len(successful_pages)} pages")
        print(f"📁 All files saved in: {output_dir}")
        
        # Create summary file
        summary_filename = f"{base_name}_extraction_summary.txt"
        summary_filepath = os.path.join(output_dir, summary_filename)
        
        with open(summary_filepath, 'w', encoding='utf-8') as f:
            f.write(f"Text Extraction Summary\n")
            f.write("=" * 50 + "\n")
            f.write(f"Source PDF: {pdf_path}\n")
            f.write(f"Page Range: {start_page + 1} to {end_page + 1}\n")
            f.write(f"Total Pages in PDF: {total_pages}\n")
            f.write(f"Successfully Processed: {len(successful_pages)} pages\n")
            f.write(f"DPI Setting: {dpi}\n")
            f.write(f"Extraction Date: {pd.Timestamp.now()}\n")
            f.write(f"Output Directory: {output_dir}\n\n")
            
            f.write("Successfully Processed Pages:\n")
            f.write("-" * 30 + "\n")
            for page in successful_pages:
                f.write(f"Page {page}\n")
            
            f.write(f"\nFiles Created:\n")
            f.write("-" * 20 + "\n")
            f.write(f"- Individual page files: {len(successful_pages)} files\n")
            f.write(f"- Combined text file: {combined_filename}\n")
            f.write(f"- Summary file: {summary_filename}\n")
        
        print(f"📋 Summary saved: {summary_filename}")
        
        return successful_pages, output_dir
    else:
        print("❌ No text was extracted from any pages!")
        return [], output_dir

In [ ]:
# 💾 Extract and save text to .txt files for backup and analysis
# This will create individual page files and a combined file

# Example 1: Save text from the same page range used for vector store
successful_pages, output_dir = save_extracted_text_to_files(
    pdf_path="Data/HSC26-Bangla1st-Paper.pdf",
    start_page=42,    # Same as vector store range
    end_page=49,      # Same as vector store range
    dpi=400,          # Same DPI for consistency
    output_dir="extracted_text_bengali"  # Custom directory name
)

print(f"\n🎉 Text extraction completed!")
print(f"📁 Files saved in: {output_dir}")
print(f"📄 Successfully processed: {len(successful_pages)} pages")

In [ ]:
# 📁 Additional utility functions for text file management

def list_extracted_text_files(output_dir: str = "extracted_text_bengali"):
    """
    List all extracted text files in the directory
    """
    import os
    
    if not os.path.exists(output_dir):
        print(f"❌ Directory {output_dir} does not exist")
        return
    
    files = [f for f in os.listdir(output_dir) if f.endswith('.txt')]
    
    if not files:
        print(f"📁 No .txt files found in {output_dir}")
        return
    
    print(f"📄 Found {len(files)} text files in {output_dir}:")
    print("=" * 60)
    
    # Separate different types of files
    page_files = [f for f in files if '_page_' in f]
    combined_files = [f for f in files if '_combined_' in f]
    summary_files = [f for f in files if '_summary' in f]
    
    if page_files:
        print(f"\n📄 Individual Page Files ({len(page_files)}):")
        for f in sorted(page_files):
            file_path = os.path.join(output_dir, f)
            size = os.path.getsize(file_path)
            print(f"  - {f} ({size:,} bytes)")
    
    if combined_files:
        print(f"\n📚 Combined Files ({len(combined_files)}):")
        for f in combined_files:
            file_path = os.path.join(output_dir, f)
            size = os.path.getsize(file_path)
            print(f"  - {f} ({size:,} bytes)")
    
    if summary_files:
        print(f"\n📋 Summary Files ({len(summary_files)}):")
        for f in summary_files:
            file_path = os.path.join(output_dir, f)
            size = os.path.getsize(file_path)
            print(f"  - {f} ({size:,} bytes)")

def read_extracted_page(page_num: int, output_dir: str = "extracted_text_bengali", show_raw: bool = False):
    """
    Read and display extracted text from a specific page
    """
    import os
    
    # Find the page file
    files = os.listdir(output_dir) if os.path.exists(output_dir) else []
    page_file = None
    
    for f in files:
        if f'_page_{page_num:03d}.txt' in f:
            page_file = f
            break
    
    if not page_file:
        print(f"❌ Page {page_num} text file not found in {output_dir}")
        return
    
    file_path = os.path.join(output_dir, page_file)
    
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    print(f"📄 Content from Page {page_num}:")
    print("=" * 60)
    
    if show_raw:
        print(content)
    else:
        # Show only cleaned text
        parts = content.split("Cleaned Text:")
        if len(parts) > 1:
            cleaned_part = parts[1].split("=" * 50)[0].strip()
            print(cleaned_part[:1000] + "..." if len(cleaned_part) > 1000 else cleaned_part)
        else:
            print(content[:1000] + "..." if len(content) > 1000 else content)

# Example usage:
# list_extracted_text_files()
# read_extracted_page(43)  # Read page 43 content